In [ ]:
###--- DATA GENERATOR ----     ####
### @@ FRACTION AT PEAK    ###
### ^^ NumCases Vs PeakDate   ###


####################################_____[[ DATA GENERERATOR ]] FOR FRACTION AT PEAK RATIO ______________________ #################################

from scipy.stats import norm
import numpy
import sklearn
from sklearn.neighbors import KernelDensity
from sklearn.utils.fixes import parse_version
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn import preprocessing
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt



def GenerateData_ratio(df_active,YColumn,Bandwidth,N_bins,LinePLotVals_Cases,LinePLotX_Cases,NoWaveRatio,boolHist):
    fig, ax = plt.subplots(3,2,figsize=(11,8))
    fig.tight_layout(pad=3.0)
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    kernel = 'epanechnikov'
    KDPCIR=["CIR= 40", "CIR= 50"]
    KDPImm=["EEscape Period - July","Escape Period - September", "Escape Period  - November"]
    legend = ["Present Rate","1.5x Present Rate","2x Present Rate"]
    
    
    
    for i in range(3):
        for j in range(2):
            df_kde=df_active.copy(deep=True)
            df_kde=df_kde[df_kde['CIR']==KDP_nu_name[j]]
            df_kde=df_kde[df_kde['KDPwsat2']==KDP_w_sat_2_name[i]]
            
            #index for storing Lineplot arrays
            plotIndex = 0
            k = 0;
            
            df_kdep = df_kde.copy(deep=True)
            

            TotalRows = df_kdep.shape[0]   ### NUmber of Rows for this Selected Scenario

            ### Remove Rows With DATE_OF_PEAK = 670
            df_kdep = df_kdep[df_kdep['Date Of Peak']!=NOWAVEPEAKDATE]

            ### Remove the rows with NUmber at Peak < 500 
#             df_kdep = df_kdep[df_kdep['Number At Peak']>200]


#                 print("Scenario : ",KDPCIR[j],"_",KDPImm[i],"_",legend[k])
#                 print("Total Rows : ", TotalRows)
#                 print("No Wave Peeks : ", df_kdep.shape[0])
#                 print("Max Peak : ", max(df_kdep['Number At Peak']) )
            ## Truncated Rows
            TruncatedRows = df_kdep.shape[0]


            hist = sns.histplot(ax=ax[i,j],data=df_kdep,x='fracAtDop',stat='probability',\
                                kde=True,label=legend[k],kde_kws={"bw_adjust":Bandwidth},
                                color=colorVal[k],bins=N_bins)


            ratio = TruncatedRows/TotalRows    #### Compute Ratio of Wave Scenarios

            ratioNoWave = 1.0 - ratio;


            if(len(hist.get_lines()) == plotIndex+1):
                ll = hist.get_lines()[0].get_data()
                LinePLotVals_Cases[i][j] = np.asarray(ll[1])
                LinePLotX_Cases[i][j] = np.asarray(ll[0])

                LinePLotVals_Cases[i][j] *=  ratio
                NoWaveRatio[i][j]         =  ratioNoWave

                plotIndex += 1

            else:     ##### No Cases Available
                print("SOMETHING WRONG")
                NoWaveRatio[i][j]        =   ratioNoWave
                LinePLotVals_Cases[i][j] =   np.zeros(len(LinePLotVals_Cases[i][j]))
                LinePLotX_Cases[i][j]    =   np.asarray(hist.get_lines()[0].get_data()[0])

#                 ###GET MAX Y VALUE for Y lim
#                maxYVal = max(LinePLotVals_Cases[i][j][k])

            if(not boolHist):
                ax[i,j].containers[0].remove() # remove the bars
            else:
                ax[i,j].legend(loc='upper right')
                handles, labels = ax[i,j].get_legend_handles_labels()


            ax[i,j].set_title(KDPImm[i]+", "+ KDPCIR[j])
            ax[i,j].set_xlabel(YColumn)
#             ax[i,j].set_xlim([1,5e6])
#                ax[i,j].set_ylim([0,maxYVal+0.05])
        
                
                

#     plt.legend(handles, labels, loc='lower center',title='Vaccination Rate',fancybox=True,ncol=3,bbox_to_anchor=(0.5,0.97))
    plt.tight_layout()

    plt.show()








In [ ]:
###--- FUNCTION DEFINITON ----     ####
### @@ CIR_RATIO_PLOTTER    ###

###################### ____________________ PLOTTING SCRIPT _______________ ####################################
####  Input parameter -- DataFrame
####  Generates the 2*3 plot for Each CIR Scenario  [Num at Peak for 3 Immune Escape Variants] [Date of Peak for Immune Escape variants]

def CIR_plotter_ratio(Y_VALS,X_VALS,PlotNameExtension):

    ## 0 - cir40, 1-cir50

    ####### -- PLotter Parameters -- ######
    plt.rcParams["text.usetex"] = False
    plt.rcParams["font.family"] = "serif"
    plt.rcParams["font.serif"] = ["Computer Modern"]
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12

    SMALL_SIZE = 12
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    ## Set plotting parameters   -- Day of Peak
    markersizep = 1.5

    ## Set plotting parameters   -- Number at Peak
    markersizep = 3.5

    xt = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75]
    xtl = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75]

    ## COMMON PLOTTING PARAMETERS
    markeveryVal = 9
    alphaVal = 0.3

   

    fig, ax = plt.subplots(3,2,figsize=(11,8),sharex=False,sharey=False)
    fig.tight_layout()

    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    marker  = ['o','h','^']
    legend = ["Age 0-11","Age 11-18","Age 18+"]
    KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
    KDPCIR = ["CIR 40", "CIR 50"]

    ## MaxValueLegend 
    MaxValueLegend = [];


    for i in range(3):
        for j in range(2):
        
            for k in range(3):
        #             print( "(i,j) = (", str(i) ,"," ,str(j),") ",   " K val : " , str(k) )
        #            sns.lineplot(ax=ax[i,j],data=LinePLotVals[i][j][k],label=KDP_c_p_name[k])
                ax[i,j].plot(X_VALS[k][i][j],Y_VALS[k][i][j],label = legend[k],linewidth=1.5,color=colorVal[k],marker=marker[k],markersize=markersizep,markevery=markeveryVal)

                ax[i,j].fill_between(X_VALS[k][i][j],Y_VALS[k][i][j],0, alpha=alphaVal, color=colorVal[k])
                MaxValueLegend.append(max(Y_VALS[k][i][j]))




            ax[i,j].legend(title='Age Groups',fancybox=True,loc='upper right')
    #         ax[i,col].set_title(KDPImm[i] +", "+ KDPCIR[j], y=10.0, pad=-14)
            ax[i,j].text(.48,.9,KDPImm[i] +", "+ KDPCIR[j],horizontalalignment='center',transform=ax[i,j].transAxes)
            ax[i,j].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,j].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
    #             ax[i,col].set_xlim()
            ax[i,j].set_ylim((0,max(MaxValueLegend)+0.05))
            ax[i,j].set_xticks(xt);
            ax[i,j].set_xticklabels([]);
            handles, labels = ax[i,j].get_legend_handles_labels()
            ax[i,j].get_legend().remove()
        
        

        
        ##YLavels
        ax[i,0].set_ylabel("""Probality of R""")

        ## X labels
    ax[2,0].set_xlabel("""Ratio at Peak (R)""")
    ax[2,0].set_xticks(xt);
    ax[2,0].set_xticklabels(xtl);
    ax[2,0].tick_params(axis='x', rotation=60);
    
            ## X labels
    ax[2,1].set_xlabel("""Ratio at Peak (R)""")
    ax[2,1].set_xticks(xt);
    ax[2,1].set_xticklabels(xtl);
    ax[2,1].tick_params(axis='x', rotation=60);
    
    fig.legend(handles, labels, loc='lower center',title='AgeGroup Ratio',fancybox=True,ncol=3,bbox_to_anchor=(0.5,0.98))
    plt.tight_layout()
    
    print("SavePlot Start");

    name = "ratio"
    name = name + "_" + PlotNameExtension
    plt.savefig(name + ".png",dpi=300,bbox_inches='tight')

    print("SavePlot End");
    plt.show()
#         plt.draw()



In [2]:
###--- PLOTTING CELL ----     ####
### @@ AGE RATIO PLOT    ###

def plotDataFrame_ratio(plotName):
#     DataFrameName = AddPeakInfo(DataFrameName,700,False)

    ## For Number of Below 11   
    LinePLotVals_11 = np.empty(shape=(3,2,200))
    LinePLotX_11 = np.empty(shape=(3,2,200))

    ## For Date of Peak
    LinePLotVals_18 = np.empty(shape=(3,2,200))
    LinePLotX_18 = np.empty(shape=(3,2,200))
    
    ## For Date of Peak
    LinePLotVals_18plus = np.empty(shape=(3,2,200))
    LinePLotX_18plus = np.empty(shape=(3,2,200))
        
    ### For Storing Ratio of No Peak in Each cases
    NoWave_11_Ratio = np.empty(shape = (3,2,1,1))
    NoWave_18_Ratio = np.empty(shape = (3,2,1,1))
    NoWave_18plus_Ratio = np.empty(shape = (3,2,1,1))
    
    
    ### Data for Nuc_11
    GenerateData_ratio(df_nucleation_11_ratio,'fracAtDop',0.4,10,LinePLotVals_11,LinePLotX_11,NoWave_11_Ratio,True)
    
    ### Data for Nuc_18
    GenerateData_ratio(df_nucleation_18_ratio,'fracAtDop',0.4,10,LinePLotVals_18,LinePLotX_18,NoWave_18_Ratio,True)
    
    ### Data for Nuc_18plus
    GenerateData_ratio(df_nucleation_18plus_ratio,'fracAtDop',0.4,10,LinePLotVals_18plus,LinePLotX_18plus,NoWave_18plus_Ratio,True)
    
    Y_VALS = []
    X_VALS = []
    
    Y_VALS.append(LinePLotVals_11)
    Y_VALS.append(LinePLotVals_18)
    Y_VALS.append(LinePLotVals_18plus)
    
    
    X_VALS.append(LinePLotX_11)
    X_VALS.append(LinePLotX_18)
    X_VALS.append(LinePLotX_18plus)
    
    
    ### PLot the values in seperate plots
    CIR_plotter_ratio(Y_VALS,X_VALS,plotName)
    

    
plotDataFrame_ratio("AgeDistribution")


NameError: name 'np' is not defined

In [ ]:
#### ---- PLOTTER ---- #####
# @@ Multiple Line plot -- @@


import matplotlib.pyplot as plt

def multipleLinePlotter(SD_Col,index):
    df_active_copy =  df_active.copy(deep=True)

    df_active_copy = df_active_copy[df_active_copy["SD"] == SD_Col]
    KDP_w_sat_2_name=["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov"]
    KDP_c_p_name=["Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent"]
    f,ax = plt.subplots(1,3,figsize=(20,8))
    X = np.linspace(300,730,431)

    ## Plots
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
    KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
    alpha1 = [0.4,0.8,1]
    linwidth = [4,3,2]
    SDtitle = ["SD-Good","SD-Bad","SD-Worse"]
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
    labels1 = []
    handles1=[]
    for j in range(len(KDP_w_sat_1_name)): 
        df_plot = df_active_copy[df_active_copy["KDPwsat2"] == KDP_w_sat_2_name[j]]

        for k in range(len(KDP_c_p_name)):
            df_plot_1 = df_plot[df_plot["Vaccine Rate"] == KDP_c_p_name[k]]
            for i in range(df_plot_1.shape[0]):
                array  =  df_plot_1.iloc[i,300:731].to_numpy()
                ax[j].plot(X,array,color=colorVal[k],linewidth=linwidth[k],label=KDP_c_p_name[k],alpha=alpha1[k])
                handles,labels = ax[j].get_legend_handles_labels()
        handles1.append(handles[0])
        handles1.append(handles[36])
        handles1.append(handles[72])
        labels1.append(labels[0])
        labels1.append(labels[36])
        labels1.append(labels[72])

        ax[j].legend(handles1,labels1,loc="upper right")
        ax[j].text(0.5,0.85, KDP_w_sat_2_name[j],horizontalalignment='center',transform=ax[j].transAxes)
        handles1 = []
        labels1 = []
        

    plt.suptitle(SDtitle[index])
    name = "multipleLine_" + SDtitle[index] + ".png"
    plt.savefig(name,dpi=300)
multipleLinePlotter("SD_RA",0)
multipleLinePlotter("SD_RAR",1)
multipleLinePlotter("SD_RARF",2)